# Web scraping stat BRK

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [2]:
url = "https://leagueoflegends.fandom.com/wiki/Blade_of_the_Ruined_King"
url

'https://leagueoflegends.fandom.com/wiki/Blade_of_the_Ruined_King'

In [3]:
response = requests.get(url)
html = BeautifulSoup(response.text, "html.parser")

## Bases stats

In [4]:
brk_stat = html.findAll("div",{"class" : "pi-data-value pi-font"})[:3]
brk_stat

[<div class="pi-data-value pi-font">+40 <a href="/wiki/Attack_damage" title="Attack damage">attack damage</a></div>,
 <div class="pi-data-value pi-font">+25% <a href="/wiki/Attack_speed" title="Attack speed">attack speed</a></div>,
 <div class="pi-data-value pi-font">+8% <a href="/wiki/Life_steal" title="Life steal">life steal</a></div>]

In [5]:
for idx,stats in enumerate(brk_stat):
    brk_stat[idx] = re.search(r'(\d+)', stats.text).group(1)

# take account of %
brk_stat = list(map(int,brk_stat))
brk_stat[1] /=100
brk_stat[2] /=100
print(brk_stat)


[40, 0.25, 0.08]


## HP % Passive

In [6]:
#brk_passives = html.findAll("div",{"class" : "pi-data-value pi-font"})[3:5]
brk_passive  = html.find("span",{"style": "white-space:pre; position:relative;", "data-game"  : "lol"})
brk_passive = brk_passive.text
brk_passive = re.search(r'(\d+)', brk_passive).group(1)
brk_passive = float(brk_passive) / 100
brk_passive

0.12

In [7]:
brk_passive_range  = html.find("span",{"style": "white-space:pre; position:relative;", "data-game"  : "lol" , "data-tip" : "Ranged"})
brk_passive_range = brk_passive_range.text
brk_passive_range = re.search(r'(\d+)', brk_passive_range).group(1)
brk_passive_range = float(brk_passive_range) / 100
brk_passive_range

0.09

## Slow passive

In [8]:
brk_3autos = html.find("span",{"style" : "position:relative; border-bottom:1px dotted; cursor:help;"})
print(brk_3autos['data-bot_values'])
print(brk_3autos['data-top_values'])

40;47;54;61;68;75;82;89;96;103
1;10;11;12;13;14;15;16;17;18


In [9]:
three_autos_lvl = brk_3autos['data-top_values'].split(";")
print(three_autos_lvl)
three_autos_dmg = brk_3autos['data-bot_values'].split(";")
print(three_autos_dmg)

three_autos_dict = {key: value for key, value in zip(three_autos_lvl, three_autos_dmg)}
print(three_autos_dict)


['1', '10', '11', '12', '13', '14', '15', '16', '17', '18']
['40', '47', '54', '61', '68', '75', '82', '89', '96', '103']
{'1': '40', '10': '47', '11': '54', '12': '61', '13': '68', '14': '75', '15': '82', '16': '89', '17': '96', '18': '103'}


## Item creation

In [10]:
%run class_item.ipynb
import copy

In [11]:
brk_dict = {"ad" : brk_stat[0], "as" : brk_stat[1], "lifesteal" : brk_stat[2]}
brk_dict['passive'] = brk_passive
brk_dict['three_autos'] = three_autos_dict
brk_dict_range = copy.deepcopy(brk_dict)
brk_dict_range['passive'] = brk_passive_range


In [12]:
BRK_melee = Item("brk", brk_dict)
BRK_range = Item("brk", brk_dict_range)

In [13]:
print(BRK_melee)
print(BRK_range)

brk = {'ad': 40, 'as': 0.25, 'lifesteal': 0.08, 'passive': 0.12, 'three_autos': {'1': '40', '10': '47', '11': '54', '12': '61', '13': '68', '14': '75', '15': '82', '16': '89', '17': '96', '18': '103'}}
brk = {'ad': 40, 'as': 0.25, 'lifesteal': 0.08, 'passive': 0.09, 'three_autos': {'1': '40', '10': '47', '11': '54', '12': '61', '13': '68', '14': '75', '15': '82', '16': '89', '17': '96', '18': '103'}}
